In [170]:
!pip install opencv-python-headless
!pip install opencv-python
!pip install scipy
!pip3 install pandas

In [311]:
import scipy.io
import cv2
import pandas as pd
import os
import random 
import math

In [352]:
def parseVideoName(video_name):
    video_name = video_name.replace('.avi', '')
    first_part,second_part = video_name.split(',')
    video_type, num_targets = first_part.rsplit('_', 1)
    videos_info = second_part.split('_')
    
    parsed_data = {
        'video_name': video_name,
        'target_name': second_part,
        'video_type': video_type,
        'num_targets': int(num_targets),
        'first_video': int(videos_info[0]),
        'second_video': int(videos_info[1]),
        'first_start_frame': int(videos_info[2]),
        'second_start_frame': int(videos_info[3]),
        'first_rotation': float(videos_info[4]),
        'second_rotation': float(videos_info[5])
    }
    
    return parsed_data
    
def generateRandomColor():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def processTrialsVideos(file_path):
    df = pd.read_csv(file_path)
    parsed_rows = []
    
    for index, row in df.iterrows():
        video_name = row['videoName']
        parsed_data = parseVideoName(video_name)
        parsed_rows.append(parsed_data)
    
    return pd.DataFrame(parsed_rows)
    
def loadMatFile():
    root_folder = 'GazeMat'
    test_subs = ['sub11', 'sub21', 'sub31', 'sub41', 'sub51']

    data = {}
    
    for sub_dir in os.listdir(root_folder):
        if sub_dir in test_subs:
            mat_file_path = os.path.join(root_folder, sub_dir, '40.mat')
            if os.path.exists(mat_file_path):
                mat_data = scipy.io.loadmat(mat_file_path)
                data[sub_dir] = mat_data["extractedData"]
            else:
                print(f"{mat_file_path} does not exist.")
    return data
    print("Data loaded successfully.")

def generateSubjectName(subIndex):
    number_part = subIndex[3:]
    modified_number = number_part[:-1] if len(number_part) > 1 else '0'
    return f'sub{modified_number}'

def getVideoInfo(video):
    fps = video.get(cv2.CAP_PROP_FPS)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # num_frames = int(mainVideo.get(cv2.CAP_PROP_FRAME_COUNT))
    return fps, width, height 

def createOutputVideo(video,name):
    output_gaze_videos = 'gazeOutputVideo'
    os.makedirs(output_gaze_videos, exist_ok=True)
    fps, width, height = getVideoInfo(video)
    
    output_gaze_videos_path = os.path.join(output_gaze_videos, name + '.avi')
    output_video = cv2.VideoWriter(output_gaze_videos_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))
    return output_video

def getSubjectColor(subject):
    if subject not in subject_colors:
        subject_colors[subject] = generateRandomColor()
    return subject_colors[subIndex]

def mapPointsOnFrame(frame):
    return round(frame * (7798 / 110))

def openVideo(video):
    trialVideo = cv2.VideoCapture(os.path.join('./videos', video.video_type, video.target_name + '.avi'))
    
    if not trialVideo.isOpened():
        raise ValueError(f"Unable to open video file: {os.path.join('./videos', video.video_type, video.target_name + '.avi')}")
    return trialVideo

def generateXPoints(x):
    scale_x = 660 / 1920
    return int(x * scale_x)

def generateYPoints(y):
    scale_y = 660 / 1080
    return int(y * scale_y)

In [353]:
### to Check data 

for index in subjects:
    x_points = subjects[index][0]  # First column for x points
    y_points = subjects[index][1]
print(check_all_elements_type(subjects['sub11'][1][0]))
xc = subjects['sub11'][0][0]
yc = subjects['sub11'][0][1]

xc = subjects['sub11'][1][0][3445][0]
yc = subjects['sub11'][1][1]

xc, yc

False


(np.float64(876.2000122070312),
 array([[296.5       ],
        [296.1000061 ],
        [295.6000061 ],
        ...,
        [465.29998779],
        [466.6000061 ],
        [467.20001221]]))

In [354]:
subjects = loadMatFile()
subject_colors = {}

In [355]:
csvSample = processTrialsVideos('./sub21.csv')

In [356]:
videoList = []
trialNumber  = 1

for index, video in csvSample.iterrows():
    try:
        
        open_video = openVideo(video)
        output_video = createOutputVideo(open_video,video.video_name)
        
        frame_idx = 0

        while True:
            ret, frame = open_video.read()
            if not ret:
                break  
            
            for subIndex in subjects:
                x_points = subjects[subIndex][trialNumber][0]
                y_points = subjects[subIndex][trialNumber][1]
                point_add = mapPointsOnFrame(frame_idx)
                    
                if 0 <= point_add < len(x_points):
                    x = x_points[point_add][0]
                    y = y_points[point_add][0] 

                    if math.isnan(x):
                        if previous_x is not None:
                            x = previous_x
                        else:
                            print(f"NaN detected for x at frame {frame_idx}, but no previous value to replace. Skipping.")
                            continue

                    if math.isnan(y):
                        if previous_y is not None:
                            y = previous_y
                        else:
                            print(f"NaN detected for y at frame {frame_idx}, but no previous value to replace. Skipping.")
                            continue
            
                    # Store the current valid value as previous
                    previous_x = x
                    previous_y = y
                    
                    new_x = generateXPoints(x)
                    new_y = generateYPoints(y)
                    color = getSubjectColor(subIndex)
                    
                    cv2.circle(frame, (new_x, new_y), 5, color, 3) 

                    cv2.putText(frame, generateSubjectName(subIndex), (new_x + 5, new_y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

            resized_frame = cv2.resize(frame, (660, 660))
            output_video.write(resized_frame)
            
            frame_idx += 1
        
        trialNumber += 1
        
    except Exception as e:
        print(f"An error occurred while processing video at index {index}: {e}")

    finally:
        # Release everything
        if 'open_video' in locals() and open_video.isOpened():
            open_video.release()
        if 'output_video' in locals():
            output_video.release()
        cv2.destroyAllWindows()
    

An error occurred while processing video at index 39: index 40 is out of bounds for axis 0 with size 40
